# GenAI Prompt Engineering Examples

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [1]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('google.cloud.bigquery', 'google-cloud-bigquery')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [2]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [3]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [4]:
REGION = 'us-central1'

# Set the BUCKET name for saving work:
BUCKET = PROJECT_ID

packages:

In [5]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import storage

import vertexai.vision_models # Imagen Models
import vertexai.preview.vision_models
import vertexai.language_models # PaLM and Codey Models
import vertexai.generative_models # for Gemini Models

import json
import io
import base64
import asyncio
import requests
import IPython
import datetime, time

In [6]:
aiplatform.__version__

'1.67.1'

clients:

In [7]:
vertexai.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

bucket = gcs.lookup_bucket(BUCKET)

---
## Vertex AI Package

With the [vertexai](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai) client there are packages for the types of data being interacted with.  There is also a higher package for preview models (not yet in GA).

> **NOTE:** In can be helpful to review the API Documentation at it's source in GitHub for up to the moment release information: [github/googleapis/python-aiplatform](https://github.com/googleapis/python-aiplatform/tree/main)

Gemini Text and Multimodal Models:
- [vertexai.generative_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.generative_models)
    - [vertexai.preview.generative_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.generative_models)

Language Models (PaLM and Codey Models):
- [vertexai.language_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models)
    - [vertexai.preview.language_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.language_models)

Vision Models (Imagen Models):
- [vertexai.vision_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.vision_models)
    - [vertexai.preview.vision_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.vision_models)


In [9]:
# Gemini Models
gemini_text = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-002")
gemini_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-vision-001")
gemini15_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.5-pro-002")
gemini_flash = vertexai.generative_models.GenerativeModel("gemini-1.5-flash-002")

# PaLM and Codey Models
text_model = vertexai.language_models.TextGenerationModel.from_pretrained('text-bison')
chat_model = vertexai.language_models.ChatModel.from_pretrained('chat-bison')
textembed_model = vertexai.language_models.TextEmbeddingModel.from_pretrained('textembedding-gecko')
codegen_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-bison')
codecomp_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-gecko')
codechat_model = vertexai.language_models.CodeChatModel.from_pretrained('codechat-bison')

# Imagen Models
imagecap_model = vertexai.vision_models.ImageCaptioningModel.from_pretrained("imagetext")
imageqna_model = vertexai.vision_models.ImageQnAModel.from_pretrained("imagetext")
imagetext_model = vertexai.vision_models.ImageTextModel.from_pretrained("imagetext")
multimodalembed_model = vertexai.vision_models.MultiModalEmbeddingModel.from_pretrained('multimodalembedding')
imagen1_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@002')
imagen2_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@005')

---
## Gemini Models

[Reference: Gemini API](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini)

### Text Prompt

In [11]:
prompts = [
    "What is a mulligan?",
    "What do you do when you have an unplayable lie in golf?"
]

In [12]:
response = gemini_text.generate_content(prompts[1])
response

candidates {
  content {
    role: "model"
    parts {
      text: "## Dealing with an Unplayable Lie in Golf\n\nWhen faced with an unplayable lie, it\'s crucial to know the various options available to you as per the Rules of Golf. Here are the steps to follow:\n\n1. **Declare the Ball Unplayable**: Announce your intention to the other players in your group.\n\n2. **Take Relief**: You have two primary choices for relief:\n\n   - **Stroke and Distance**: This allows you to drop a ball within two club-lengths of the unplayable lie, no closer to the hole, and incur a one-stroke penalty. You may drop in any direction within this area, even in a better lie.\n   - **Back-on-the-Line Relief**: Drop a ball behind the point where your ball last crossed the margin of the unplayable area, keeping that point directly between the hole and the spot where you drop the ball. You can drop within two club-lengths of that point, no closer to the hole, and incur a one-stroke penalty.\n\n3. **Play the Dro

In [13]:
print(response.text)

## Dealing with an Unplayable Lie in Golf

When faced with an unplayable lie, it's crucial to know the various options available to you as per the Rules of Golf. Here are the steps to follow:

1. **Declare the Ball Unplayable**: Announce your intention to the other players in your group.

2. **Take Relief**: You have two primary choices for relief:

   - **Stroke and Distance**: This allows you to drop a ball within two club-lengths of the unplayable lie, no closer to the hole, and incur a one-stroke penalty. You may drop in any direction within this area, even in a better lie.
   - **Back-on-the-Line Relief**: Drop a ball behind the point where your ball last crossed the margin of the unplayable area, keeping that point directly between the hole and the spot where you drop the ball. You can drop within two club-lengths of that point, no closer to the hole, and incur a one-stroke penalty.

3. **Play the Dropped Ball**: Once dropped, the ball is in play. You cannot take another penalty 

In [14]:
IPython.display.Markdown(response.text)

## Dealing with an Unplayable Lie in Golf

When faced with an unplayable lie, it's crucial to know the various options available to you as per the Rules of Golf. Here are the steps to follow:

1. **Declare the Ball Unplayable**: Announce your intention to the other players in your group.

2. **Take Relief**: You have two primary choices for relief:

   - **Stroke and Distance**: This allows you to drop a ball within two club-lengths of the unplayable lie, no closer to the hole, and incur a one-stroke penalty. You may drop in any direction within this area, even in a better lie.
   - **Back-on-the-Line Relief**: Drop a ball behind the point where your ball last crossed the margin of the unplayable area, keeping that point directly between the hole and the spot where you drop the ball. You can drop within two club-lengths of that point, no closer to the hole, and incur a one-stroke penalty.

3. **Play the Dropped Ball**: Once dropped, the ball is in play. You cannot take another penalty to re-drop it.

4. **Specific Unplayable Lies**: Certain situations have specific relief options:
   - **Embedded Ball**: If your ball is embedded in its own pitch-mark, you can take relief under Rule 16.3b, either without penalty by placing a ball within one club-length of and not nearer the hole than the original position, or with a one-stroke penalty under the two club-lengths options mentioned above.
   - **Ball Lost or Out of Bounds**: If you cannot find your ball or it's deemed out of bounds, proceed as if it were unplayable, incurring the applicable stroke penalty and taking relief accordingly.

**Additional Tips**:

* It's important to carefully assess the situation before declaring a ball unplayable. Sometimes, a seemingly difficult lie can be played successfully.
* When taking relief, choose a spot that gives you the best possible opportunity to play your next shot.
* Remember to mark the position of your original ball before taking relief.

By understanding the rules and taking advantage of the relief options available, you can minimize the damage of an unplayable lie and continue your round effectively.

#### Streaming Response

In [15]:
for response in gemini_text.generate_content(prompts[1], stream = True):
    print(response.text)

There
 are a few options available when you find yourself with an unplayable lie in golf:


**Take an unplayable lie penalty:** This is the most common option. You will
 add one stroke to your score and drop the ball in a designated relief area. The relief area is typically two club-lengths from the spot where the unplayable lie occurred
, no closer to the hole. You must drop the ball within the relief area, not closer to the hole than the original position.

**Hit a provisional
 ball:** Before deciding the lie is unplayable, you can hit a provisional ball from a different location (often the fairway or rough) where you have a playable lie. If you find the original ball and confirm the lie is unplayable, you can
 take a one-stroke penalty and play the provisional ball. If you find the original ball and it's playable, you can continue playing it and discard the provisional ball.

**Declare the ball lost:** If you are unable to find the ball or
 believe it is lost, you can declare it los

#### Async Response

The client has built in method for awaitable responses that make it easy to make asynchronous request.

> For detailed coverage and examples of asynchronous call to these API's, scaling, error handling, and managing fail over regions check out this notebook in the same folder: [Python Asynchronous API Calls](./Python%20Asynchronous%20API%20Calls.ipynb)

In [16]:
response = await gemini_text.generate_content_async(prompts[1])
#print(response.text)
IPython.display.Markdown(response.text)

## Unplayable Lies in Golf

An unplayable lie in golf can be frustrating, but don't worry, there are options! Here's what you can do:

**1. Take a Penalty Stroke:**

This is the simplest option. You can take a one-stroke penalty and drop the ball within two club lengths of the original spot, no closer to the hole. Keep in mind that these two club lengths should be measured in any direction, except towards the hole.

**2. Play the Ball as it Lies:**

While challenging, you can attempt to play the ball as it lies. This could be a good option if you're confident in your ability to hit a difficult shot or if the penalty stroke would put you in an even worse position.

**3. Take Lateral Relief:**

If your ball is in a penalty area (water hazard or out-of-bounds), you can take lateral relief. This means dropping the ball within two club lengths of the point where the ball crossed the margin of the penalty area, but not closer to the hole. There is a two-stroke penalty for taking lateral relief.

**4. Provisional Ball:**

Before hitting your original shot, you can hit a provisional ball in case your original shot ends up being unplayable. You can then play the provisional ball without penalty if your original shot is deemed unplayable.

**Additional Considerations:**

* Always check the local rules before determining how to proceed with an unplayable lie.
* Consider the conditions and your own skill level when choosing the best option for your situation.
* Remember to inform your playing partners and marker when taking relief or playing a provisional ball.

Here are some helpful resources:

* **USGA Rule 19:** https://www.usga.org/content/dam/usga/pdf/2023/rule-books/rules-of-golf.pdf
* **R&A Rule 19:** https://www.randa.org/en/rules-and-amateur-status/rules-of-golf
* **PGA Instruction on Unplayable Lie:** https://www.pga.com/practice/golf-tips/how-to-play-an-unplayable-lie

I hope this helps! Let me know if you have any other questions about golf or anything else.